### Dahboard:Deforestation and Rainfall stats for Indian Subcontinent

#### Poonam Vyas MT2030

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input,Output
import json
states=json.load(open("states_india.geojson",'r'))
#states

In [ ]:
external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css']
app=dash.Dash(__name__, external_stylesheets=external_stylesheets)
d={}

df=pd.read_csv("Data.csv")



da=pd.read_csv("Rain.csv")

da=da[:4115]
for i in states['features']:
    i['id']=i['properties']['state_code']
    d[i["properties"]["st_nm"]]=i["id"]
    
df["id"]=df["place"].apply(lambda x:d[x])
da["id"] = da["State"].apply(lambda x: d[x])
da['Year']=da['Year'].astype(int)
available_indicators = da['Year'].unique()

In [ ]:
da.head()

In [ ]:
fig= px.choropleth(
      df,
      locations="id",
      geojson=states,
      color="deforestation(in Ha)",
      hover_name="place",
      scope="asia",
      color_continuous_scale='deep')
      
fig.update_geos(fitbounds="locations")
fig.update_layout(title_text='Deforestation by region')

jpg= px.choropleth(
    df,
    locations="id",
    geojson=states,
    color="rainfall(in mm)",
    hover_name='place',
    scope='asia',
    color_continuous_scale='matter')

jpg.update_geos(fitbounds="locations")
jpg.update_layout(title_text='Rainfall by region')



In [ ]:
smg=go.Figure(data=[
                go.Bar(name='rainfall', x= df['place'], y=df['rainfall(in mm)']),
                go.Bar(name='deforestation',x= df.place, y=df['deforestation(in Ha)'])
])

smg.update_layout(barmode="group")
smg.update_layout(title_text='Rainfallvs Deforestation')

In [ ]:
pop = px.line(da, x=da['Year'], y=da['Annual Rainfall'], color='State', symbol="State")
pop.show()
pop.update_layout(title_text='Annual Rainfall by region')

In [ ]:
cop=px.histogram(da, x="Annual Rainfall", color="State")
cop.update_layout(title_text='Annual Rainfall by region histogram')
cop.show()

In [ ]:
bob = px.histogram(da, x="DEF", color="State")
bob.update_layout(title_text='Area under deforestation by region histogram')
bob.show()

In [ ]:
app.layout= html.Div(
        html.Div([
            html.Div([
                 html.H1('Dashboard:Deforestation and Rainfall Stats for the Indian Subcontinent',style={'text-align':'center','colour':'green'},),
                 html.H2('Interactive dashboard using Plotly Dash',style={'text-align':'center','colour':'green'}), 
                
                 html.Br(),
                 html.Br(),
                 html.H2('Choropleths that display the Statewise Rainfall, Deforestation',style={'text-align':'left','colour':'#97E0C0'}),
                    ],className="row"
            ),
            
            html.Div([                
                html.Div([
                    dcc.Graph(
                        id='a',
                        figure=fig)
                    ],className='six columns'
                ),
                
                html.Div([
                    dcc.Graph(
                        id='b',
                        figure=jpg)
                    ], className="six columns"
                )
                ], className="row"
            ),
            
            html.Br(),
            html.Div([
                html.H2('Choropleth for Annual Rainfall,Deforestation,Tree cover for the selected year',style={'text-align':'Left','colour':'green'}),
                html.Div([
                    dcc.Dropdown(id="slct_year",
                         options=[
                             {'label': i, 'value': i} for i in available_indicators

                             ],
                         multi=False,
                         value=1901,
                         style={'width': "40%"}
                         ),

            html.Div(id='output_container', children=[]),
            html.Br(),                   
            
            dcc.Graph(id='my_bee_map1', figure={}),
            dcc.Graph(id='my_bee_map2', figure={}),
            dcc.Graph(id='my_bee_map3', figure={})
            ],className="six columns"
            ),
            html.Div([
                html.H2('Scatterplot for Annual Rainfall and Deforestation for the selected year',style={'text-align':'Left','colour':'green'}),
                dcc.Dropdown(id="slct_year2",
                         options=[
                             {'label': i, 'value': i} for i in available_indicators

                             ],
                         multi=False,
                         value=1901,
                         style={'width': "40%"}
                         ),

            html.Div(id='output_container2', children=[]),
            html.Br(),                  
                dcc.Graph(id='the_graph1',figure={}),
                dcc.Graph(id='the_graph2',figure={})
            ],className="six columns"
            ),
             
            html.Div([
                html.H2('Pie chart for Annual Rainfall and Deforestation for the selected year ',style={'text-align':'Left','colour':'green'}),
                dcc.RadioItems(id="radio2",
                         options=[
                             {'label': i, 'value': i} for i in available_indicators

                             ],
                         value=2010,
                         style={'width': "40%"}
                         ),

            html.Div(id='output_container3', children=[]),
            html.Br(),                  
                dcc.Graph(id='the_graph3',figure={}),
                dcc.Graph(id='the_graph4',figure={})
                    ],className="six columns"
                    )
            
            ], className="row"
            ),
           html.Div([
                 html.H2('Histogram for Annual Rainfall for various states',style={'text-align':'left','colour':'green'}),
                dcc.Graph(id='j',figure=cop)
            ],className="row"),
           html.Div([
                 html.H2('Histogram representing area under deforestation for various states',style={'text-align':'left','colour':'green'}),
                dcc.Graph(id='k',figure=bob)
            ],className="row"),
           html.Div([
                 html.H2('Line chart representing Rainfall for various states for various years',style={'text-align':'left','colour':'green'}),
                dcc.Graph(id='h',figure=pop)
            ],className="row"),
           html.Div([
                 html.H2('Bar chart representing Rainfall and Deforestation',style={'text-align':'left','colour':'green'}),
                dcc.Graph(id='p',figure=smg)
            ],className="row")
        ])
)

In [ ]:
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='my_bee_map1', component_property='figure'),
     Output(component_id='my_bee_map2', component_property='figure'),
     Output(component_id='my_bee_map3', component_property='figure')],
    [Input(component_id='slct_year', component_property='value')]
)
def update_graph(option_slctd):
    print('The year chosen by user was:',option_slctd)
    print(type(option_slctd))

    container = "The year chosen by user was: {}".format(option_slctd)
    
    dff = da.copy()
    dff = dff[dff["Year"] == option_slctd]
    
    # Plotly Express
    fig1=px.choropleth(
        dff,
        locations='id',
        geojson=states,
        color='Annual Rainfall',
        scope='asia',
        hover_name="State",
        hover_data=['State'],
        color_continuous_scale='matter')
            
    
    fig1.update_geos(fitbounds="locations", visible=False)
    fig1.update_layout(title_text='Annual Rainfall by region for selected year')
    
    fig2=px.choropleth(
      dff,
      locations="id",
      geojson=states,
      color="DEF",
      hover_name="State",
      hover_data=['State'],
      scope="asia",
      color_continuous_scale='deep')
    
    fig2.update_geos(fitbounds="locations", visible=False)
    fig2.update_layout(title_text='Deforested area by region for selected year')
    
    fig9=px.choropleth(
      dff,
      locations="id",
      geojson=states,
      color="TREE COVER",
      hover_name="State",
      hover_data=['State'],
      scope="asia",
      color_continuous_scale='Greens')
      
    fig9.update_geos(fitbounds="locations")
    fig9.update_layout(title_text='Tree cover by region for selected year')
    
    return container, fig1, fig2,fig9

In [ ]:
@app.callback(
    [Output(component_id='output_container2', component_property='children'),
     Output(component_id='the_graph1', component_property='figure'),
     Output(component_id='the_graph2', component_property='figure')],
    [Input(component_id='slct_year2', component_property='value')]
)
def update_graph2(option_slctd):
    print('The year chosen by user was:',option_slctd)
    print(type(option_slctd))

    container2 = "The year chosen by user was: {}".format(option_slctd)
    
    dss = da.copy()
    dss = dss[dss["Year"] == option_slctd]
    
    scatterplot1= px.scatter(
        data_frame=dss,
        x="State",
        y="Annual Rainfall",
        color="State",
        size='Year',
        hover_data=['State'],
        height=550
    )
    scatterplot1.update_traces(textposition='top center')
    
    scatterplot2= px.scatter(
        data_frame=dss,
        x="State",
        y="DEF",
        color="State",
        size='Year',
        hover_data=['State'],
        height=550
    )
    scatterplot2.update_traces(textposition='top center')
    
       
    return container2, scatterplot1, scatterplot2

In [ ]:
@app.callback(
    [Output(component_id='output_container3', component_property='children'),
     Output(component_id='the_graph3', component_property='figure'),
     Output(component_id='the_graph4', component_property='figure')],
    [Input(component_id='radio2', component_property='value')]
)
def update_graph3(option_slctd):
    print('The year chosen by user was:',option_slctd)
    print(type(option_slctd))

    container3 = "The year chosen by user was: {}".format(option_slctd)
    
    dss1 = da.copy()
    dss1 = dss1[dss1["Year"] == option_slctd]
    
     
    fig3=px.pie(dss1,values=dss1['Annual Rainfall'],names=dss1['State'],title='Annual rainfall pie chart for states for said year')
    fig4=px.pie(dss1,values=dss1['DEF'],names=dss1['State'],title='Deforestation pie chart for states for said year')
           
    return container3, fig3,fig4

In [ ]:
if __name__ == '__main__':
    app.run_server(port=4050,use_reloader=False)

### References:-
https://plotly.com/dash/

https://www.youtube.com/watch?v=hSPmj7mK6ng

https://plotly.com/graphing-libraries/